# Data Exploration

In [1]:
import numpy as np 
import pandas as pd 
import os

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [5]:
test["id"][0]

0

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 102.0+ KB


In [8]:
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [9]:
train[train['target']==0]['text']

15                                         What's up man?
16                                          I love fruits
17                                       Summer is lovely
18                                      My car is so fast
19                           What a goooooooaaaaaal!!!!!!
20                                 this is ridiculous....
21                                      London is cool ;)
22                                            Love skiing
23                                  What a wonderful day!
24                                               LOOOOOOL
25                         No way...I can't eat that shit
26                                  Was in NYC last week!
27                                     Love my girlfriend
28                                              Cooool :)
29                                     Do you like pasta?
30                                               The end!
32      We always try to bring the heavy. #metal #RT h...
34            

In [10]:
train['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [11]:
train[(train['target']==0) & (pd.notna(train['keyword']))].head(10)

,id,keyword,location,text,target
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
41,61,ablaze,NaN,on the outside you're ablaze and alive\nbut yo...,0
42,62,ablaze,milky way,Had an awesome time visiting the CFC head offi...,0
43,63,ablaze,NaN,SOOOO PUMPED FOR ABLAZE ???? @southridgelife,0
44,64,ablaze,NaN,I wanted to set Chicago ablaze with my preachi...,0


In [12]:
len(train['keyword'].unique())

222

In [13]:
len(train['location'].unique())

3342

# Preprocessing

## Normalize the text

In [14]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('all')

/srv/conda/envs/saturn/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/srv/conda/envs/saturn/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/jovyan/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package bi

True

In [15]:
import re

replacement_patterns = [
    #match url (i.e: https://t.co/5tF5G9VKtq)
    (r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '<url>'),

    #match user (i.e: @cerpintor )
    (r'@\w+', '<user>'),

    #Replace "&..." with ''
    (r'&\w+', '')
]

class RegexReplacer(object):
    def __init__(self, patterns = replacement_patterns):
        self.patterns = [(re.compile(regrex),repl) for (regrex, repl) in
                        patterns]
    
    #Replace the words that match the patterns with replacement words
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s

In [16]:
tknz = TweetTokenizer()
replacer = RegexReplacer()
stopwords = set(stopwords.words('english'))
punc = string.punctuation

In [17]:
#function to process the text
def normalize(doc):
    
    for i in range(len(doc)):
        
        #Tokenize with replacement
        doc[i] = tknz.tokenize(replacer.replace(doc[i]))
        
        #Filter stopwords, punctuations, and lowercase
        doc[i] = [w.lower() for w in doc[i] if w not in punc and w not in stopwords]
    
        #Stem words
        
        lemmatizer = WordNetLemmatizer()
        
        doc[i] = [lemmatizer.lemmatize(w, pos='v') for w in doc[i]]
        
        
        #concat
        doc[i] = ' '.join(w for w in doc[i])
        
    return doc

In [18]:
text_train = normalize(train['text'])

/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipyke

In [19]:
text_test = normalize(test['text'])

/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipyke

In [20]:
text_train

0       our deeds reason #earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       all residents ask shelter place notify officer...
3       13,000 people receive #wildfires evacuation or...
4       just get send photo ruby #alaska smoke #wildfi...
5       #rockyfire update california hwy 20 close dire...
6       #flood #disaster heavy rain cause flash flood ...
7                       i'm top hill i see fire woods ...
8       there's emergency evacuation happen build acro...
9                        i'm afraid tornado come area ...
10                         three people die heat wave far
11      haha south tampa get flood hah wait a second i...
12      #raining #flooding #florida #tampabay #tampa 1...
13                    #flood bago myanmar #we arrive bago
14         damage school bus 80 multi car crash #breaking
15                                             what's man
16                                           i love fruit
17            

## Split the training and test set

In [124]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_train, train['target'], test_size=0.4, random_state=0)

# Model Training

## Tf-Idf

- Apply Linear Support Vector Classification for classification
- Use GridSearchCV to find the best C parameter for the SVC
- Convert raw text to a matrix of TF-IDF features
- Predict with Tf-Idf representation of words

In [23]:
import csv
import argparse
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score
from sklearn import metrics, preprocessing
import numpy as np

from sklearn import svm, datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


parameters = {'C': [.05, .12, .25, .5, 1, 2, 4]}
svr = svm.LinearSVC(class_weight='balanced')
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)

#tf-idf
tfidf_vectorizer=TfidfVectorizer(use_idf=True)

#just send in all your docs here
tfidf_vectorizer_train=tfidf_vectorizer.fit_transform(X_train)

tfidf_vectorizer_test=tfidf_vectorizer.transform(X_test)

grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(tfidf_vectorizer_train,y_train)

labels_pred = grid.predict(tfidf_vectorizer_test)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1471  291]
 [ 332  952]]
              precision    recall  f1-score   support

           0       0.82      0.83      0.83      1762
           1       0.77      0.74      0.75      1284

    accuracy                           0.80      3046
   macro avg       0.79      0.79      0.79      3046
weighted avg       0.79      0.80      0.79      3046



"\n  mlflow.log_param('C', grid.best_params_)\n  mlflow.log_metric('f1', metrics.f1_score(y_test, labels_pred))\n\n  mlflow.sklearn.log_model(grid, 'tfidf')\n"

## Tf-Idf N-grams

N-gram is a contiguous sequence of n items from a given sample of text or speech. The items can be characters or words. I experiment with both characters and words with n-grams of different sizes

### Characters

In [139]:
#analyzer = 'char', ngram_range = (3,6): create tf-idf with n-gram of 3-6 characters
#alternative: ngrams_range = (3,3), (4,4), etc
tfidf_vectorizer_c=TfidfVectorizer(use_idf=True, analyzer = 'char', ngram_range = (3,6))

#just send in all your docs here
tfidf_vectorizer_train_c=tfidf_vectorizer_c.fit_transform(X_train)

tfidf_vectorizer_test_c=tfidf_vectorizer_c.transform(X_test)

grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(tfidf_vectorizer_train_c,y_train)

labels_pred = grid.predict(tfidf_vectorizer_test_c)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1486  276]
 [ 330  954]]
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1762
           1       0.78      0.74      0.76      1284

    accuracy                           0.80      3046
   macro avg       0.80      0.79      0.79      3046
weighted avg       0.80      0.80      0.80      3046



### Words

In [140]:
#analyzer = 'word', ngram_range = (3,6): create tf-idf with n-gram of 1-3 words
tfidf_vectorizer_w=TfidfVectorizer(use_idf=True, analyzer = 'word', ngram_range = (1,3))

#just send in all your docs here
tfidf_vectorizer_train_w=tfidf_vectorizer_w.fit_transform(X_train)

tfidf_vectorizer_test_w=tfidf_vectorizer_w.transform(X_test)

grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(tfidf_vectorizer_train_w,y_train)

labels_pred = grid.predict(tfidf_vectorizer_test_w)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1445  317]
 [ 319  965]]
              precision    recall  f1-score   support

           0       0.82      0.82      0.82      1762
           1       0.75      0.75      0.75      1284

    accuracy                           0.79      3046
   macro avg       0.79      0.79      0.79      3046
weighted avg       0.79      0.79      0.79      3046



### Concatenate Characters and Words

In [146]:
from scipy.sparse import hstack

tfidf_vectorizer_train_concat = hstack([tfidf_vectorizer_train_c, tfidf_vectorizer_train_w])
tfidf_vectorizer_test_concat = hstack([tfidf_vectorizer_test_c, tfidf_vectorizer_test_w])

grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(tfidf_vectorizer_train_concat,y_train)

labels_pred = grid.predict(tfidf_vectorizer_test_concat)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))


[[1481  281]
 [ 328  956]]
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1762
           1       0.77      0.74      0.76      1284

    accuracy                           0.80      3046
   macro avg       0.80      0.79      0.79      3046
weighted avg       0.80      0.80      0.80      3046



## TFIDF selecting k best

- Apply SelectKBest to select the 2000 best features (words)


In [24]:
from sklearn.feature_selection import SelectKBest, chi2
import scipy

In [25]:
feats = SelectKBest(chi2, k = 2000)
feats.fit(tfidf_vectorizer_train, y_train)

#Convert sparse matrix to dense matrix 
tfidf_vectorizer_train = scipy.sparse.csr_matrix.todense(tfidf_vectorizer_train)
tfidf_vectorizer_test = scipy.sparse.csr_matrix.todense(tfidf_vectorizer_test)

#Indices of best features
words_ind = feats.get_support(indices=True)

#Select the words within the list of best features
tfidf_vectorizer_train_best = np.array([tfidf_vectorizer_train[:, ind] for ind in words_ind])[:,:, 0].T
tfidf_vectorizer_test_best = np.array([tfidf_vectorizer_test[:, ind] for ind in words_ind])[:,:,0].T
tfidf_vectorizer_train_best.shape

labels_pred = grid.predict(tfidf_vectorizer_test_best)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1451  311]
 [ 352  932]]
              precision    recall  f1-score   support

           0       0.80      0.82      0.81      1762
           1       0.75      0.73      0.74      1284

    accuracy                           0.78      3046
   macro avg       0.78      0.77      0.78      3046
weighted avg       0.78      0.78      0.78      3046



"\n  mlflow.log_param('C', grid.best_params_)\n  mlflow.log_metric('f1', metrics.f1_score(y_test, labels_pred))\n\n  mlflow.sklearn.log_model(grid, 'tfidf-kbest')\n"

## Concatenate TFIDF and K-best from TFIDF

In [166]:
concat_train = hstack([tfidf_vectorizer_train, tfidf_vectorizer_train_best])
concat_test = hstack([tfidf_vectorizer_test, tfidf_vectorizer_test_best])

In [150]:
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(concat_train,y_train)

labels_pred = grid.predict(concat_test)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1503  259]
 [ 320  964]]
              precision    recall  f1-score   support

           0       0.82      0.85      0.84      1762
           1       0.79      0.75      0.77      1284

    accuracy                           0.81      3046
   macro avg       0.81      0.80      0.80      3046
weighted avg       0.81      0.81      0.81      3046



## Binary

Convert a collection of text documents to a matrix of token counts. All non zero counts are set to 1. Since tweets are short, binary may be a better choice than integer count

In [176]:
from sklearn.feature_extraction.text import CountVectorizer
import scipy

vectorizer = CountVectorizer(binary=True, stop_words=stopwords)
bin_train = scipy.sparse.csr_matrix.todense(vectorizer.fit_transform(X_train))
bin_test = scipy.sparse.csr_matrix.todense(vectorizer.transform(X_test))

In [177]:
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(bin_train,y_train)

labels_pred = grid.predict(bin_test)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1510  252]
 [ 343  941]]
              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1762
           1       0.79      0.73      0.76      1284

    accuracy                           0.80      3046
   macro avg       0.80      0.79      0.80      3046
weighted avg       0.80      0.80      0.80      3046



## Binary selecting k best

In [178]:
feats = SelectKBest(chi2, k = 1000)
feats.fit(bin_train, y_train)

SelectKBest(k=1000, score_func=<function chi2 at 0x7f74ef696510>)

In [179]:
words_ind = feats.get_support(indices=True)

In [180]:
bin_train_best = np.array([bin_train[:, ind] for ind in words_ind])[:,:, 0].T
bin_test_best = np.array([bin_test[:, ind] for ind in words_ind])[:,:,0].T

In [181]:
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(bin_train_best,y_train)

labels_pred = grid.predict(bin_test_best)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1486  276]
 [ 366  918]]
              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1762
           1       0.77      0.71      0.74      1284

    accuracy                           0.79      3046
   macro avg       0.79      0.78      0.78      3046
weighted avg       0.79      0.79      0.79      3046



## Concatenate Binary and K-best from Binary

In [163]:
bin_train.shape

(4567, 9681)

In [164]:
bin_train_best.shape

(4567, 1000)

In [168]:
bin_concat_train = np.zeros((bin_train.shape[0], bin_train.shape[1] + bin_train_best.shape[1]))
bin_concat_test = np.zeros((bin_test.shape[0], bin_test.shape[1] + bin_test_best.shape[1]))
bin_concat_sum = np.zeros((bin_sum.shape[0], bin_sum.shape[1] + bin_sum_best.shape[1]))

bin_concat_train[:,0:bin_train.shape[1]] = bin_train
bin_concat_train[:, bin_train.shape[1]:] = bin_train_best

bin_concat_test[:, 0:bin_test.shape[1]] = bin_test
bin_concat_test[:, bin_test.shape[1]:] = bin_test_best

In [169]:
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(bin_concat_train,y_train)

labels_pred = grid.predict(bin_concat_test)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1501  261]
 [ 340  944]]
              precision    recall  f1-score   support

           0       0.82      0.85      0.83      1762
           1       0.78      0.74      0.76      1284

    accuracy                           0.80      3046
   macro avg       0.80      0.79      0.80      3046
weighted avg       0.80      0.80      0.80      3046



## Word2Vec

- Word2Vec is a group of related models that are used to produce word embeddings. These models are shallow, two-layer neural networks that are trained to reconstruct linguistic contexts of words.
- Use Gensim to load pretrained Twitter word vectors from Glove (Global Vectors for Word Representation) then convert GloVe vectors into the word2vec

In [38]:
import gensim
from gensim.models import KeyedVectors

/srv/conda/envs/saturn/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [39]:
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

In [46]:
path_glove = os.path.abspath('glove.twitter.27B.200d.txt')
path_w2v = os.path.abspath('glove.twitter.27B.200d_w2v.txt')

In [47]:
path_w2v

'/home/jovyan/project/Real or Not/glove.twitter.27B.200d_w2v.txt'

In [48]:
'''glove_file = datapath(path_glove)
tmp_file = get_tmpfile(path_w2v)

_ = glove2word2vec(glove_file, tmp_file)'''

'glove_file = datapath(path_glove)\ntmp_file = get_tmpfile(path_w2v)\n\n_ = glove2word2vec(glove_file, tmp_file)'

In [49]:
path = os.path.abspath('glove.twitter.27B.200d_w2v.txt')

In [50]:
model_w2v = KeyedVectors.load_word2vec_format(path, binary=False)

In [51]:
from sklearn.feature_selection import SelectKBest, chi2

- Use SelectKBest to find the best features. Then find the w2v embedding of these best features
- Word2Vec represents vectors of words. In order to find the vector representations of documents, we find the word2vec embedding of important words in one document then take the mean of all of them.

In [52]:
def build_vector(t_train, best_words):
    
    w2v = []
    
    for doc in t_train:
                   
        mean = 0
        count = 0
        for word in doc.split():
            if word in best_words:
                try:
                    mean += np.reshape(model[word], (1, 200))
                    count +=1
                except:
                    pass
        try:
            w2v.append(mean/count)
        except:
            w2v.append(0)
                   
    return w2v



In [53]:
feats = SelectKBest(chi2, k = 1000)
feats.fit(tfidf_vectorizer_train, y_train)

SelectKBest(k=1000, score_func=<function chi2 at 0x7f74ef696510>)

In [54]:
best = feats.get_support(indices=True)
train_best = np.array([tfidf_vectorizer_train[:,ind] for ind in best])
test_best = np.array([tfidf_vectorizer_test[:,ind] for ind in best])


In [55]:
words = list(tfidf_vectorizer.vocabulary_.keys())

best_words = np.array([words[ind] for ind in best])

In [56]:
train_w2v = np.array(build_vector(X_train, best_words)) #temporal vectors, to then reshape them
test_w2v = np.array(build_vector(X_test, best_words))
w2v_train = np.zeros((len(train_w2v), 200)) #reshaped matrices with the information obtained
w2v_test = np.zeros((len(test_w2v), 200))
for ind in range(len(train_w2v)):
    w2v_train[ind, :] = train_w2v[ind]
    
for ind in range(len(test_w2v)):
    w2v_test[ind, :] = test_w2v[ind]
print(w2v_train.shape)
print(w2v_test.shape)

(4567, 200)
(3046, 200)


In [57]:
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(w2v_train,y_train)

labels_pred = grid.predict(w2v_test)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1762    0]
 [1284    0]]
              precision    recall  f1-score   support

           0       0.58      1.00      0.73      1762
           1       0.00      0.00      0.00      1284

    accuracy                           0.58      3046
   macro avg       0.29      0.50      0.37      3046
weighted avg       0.33      0.58      0.42      3046



/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 score is low. This maybe because w2v mainly represents of semantic of words. Maybe combining sematic representations and bag of words will give a better result?

## W2V concatenated with binary

In [58]:
w2v_bin_train = np.zeros((w2v_train.shape[0], w2v_train.shape[1] + bin_train.shape[1]))
w2v_bin_test = np.zeros((w2v_test.shape[0], w2v_test.shape[1] + bin_test.shape[1]))

w2v_bin_train[:, :w2v_train.shape[1]] = w2v_train
w2v_bin_train[:, w2v_train.shape[1]:] = bin_train

w2v_bin_test[:, :w2v_test.shape[1]] = w2v_test
w2v_bin_test[:, w2v_test.shape[1]:] = bin_test

In [59]:
grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)
grid.fit(w2v_bin_train,y_train)

labels_pred = grid.predict(w2v_bin_test)

print(confusion_matrix(y_test, labels_pred))
print(metrics.classification_report(y_test, labels_pred))

[[1510  252]
 [ 343  941]]
              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1762
           1       0.79      0.73      0.76      1284

    accuracy                           0.80      3046
   macro avg       0.80      0.79      0.80      3046
weighted avg       0.80      0.80      0.80      3046



# Neural Network 

- Use PyTorch to create neural network model

In [60]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [61]:
class Model(nn.Module):
    def __init__(self, inp):
        super(Model, self).__init__()  
        #Define the layers
        self.Layer1 = nn.Linear(inp, 4000)
        self.Layer2 = nn.Linear(4000, 1000)
        self.Layer3 = nn.Linear(1000, 100)
        self.Layer4 = nn.Linear(100, 2)
    def forward(self, x):
        x = F.relu(self.Layer1(x))
        x = F.relu(self.Layer2(x))
        x = F.relu(self.Layer3(x))
        m = nn.Dropout(p=0.5)
        x = m(x)
        x = self.Layer4(x)
        return F.log_softmax(x,1)

In [121]:
def get_f1(X, y_real, model):
    y_pred = []
    for x in X:
        y_pred.append(model(x.cuda()).argmax(1).cpu().detach())
    y_pred = torch.cat(y_pred).cpu().numpy()
    
    return metrics.f1_score(y_true=y_real.cpu().numpy(), y_pred=y_pred)

In [63]:
def train_model(model, epochs, Xtrain, ytrain, X_val, y_val):
    LOSS = []
    X_train = Xtrain.cuda()
    y_train = ytrain.cuda()
    for e in range(epochs):
        for i, (x, y) in enumerate(zip(X_train, y_train)):     

            y_pred = model(x)
            loss = criterio(y_pred, y)
            loss.backward()
            optimizer.step()

            LOSS.append(loss.item())
            if i%100==0:
                with torch.no_grad():
                    f1 = get_f1(X_val, y_val.cpu(), model)
                print('Epoch: %d \t Batch: %d \t Loss: %.10f \t F1_val: %.10f'%(e,i, torch.tensor(LOSS[-100:]).mean(), f1))
    return LOSS

In [182]:
#Split the training and test set to batch of 50
batch = 50
net_train = torch.from_numpy(bin_train[0:4550,:]).float()
net_train = net_train.view(-1, batch, net_train.shape[1]).float()

net_test = torch.from_numpy(bin_test[:3000,:]).float()
net_test = net_test.view(-1, batch, net_test.shape[1]).float()


Y_train = torch.tensor(list(y_train)[:4550])
Y_train = Y_train.view(-1, batch)

Y_test = torch.tensor(list(y_test)[:3000])

In [65]:
device = torch.device("cuda:0")
model = Model(net_train.shape[2]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterio  = nn.NLLLoss()

In [66]:
LOSS = train_model(model, 50, net_train, Y_train, net_test, Y_test)

/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 0 	 Batch: 0 	 Loss: 0.6908555627 	 F1_val: 0.0000000000
Epoch: 1 	 Batch: 0 	 Loss: 0.6593646407 	 F1_val: 0.6910435497
Epoch: 2 	 Batch: 0 	 Loss: 0.7313510180 	 F1_val: 0.7044734061
Epoch: 3 	 Batch: 0 	 Loss: 0.6811176538 	 F1_val: 0.7223183391
Epoch: 4 	 Batch: 0 	 Loss: 0.6168192029 	 F1_val: 0.7201350781
Epoch: 5 	 Batch: 0 	 Loss: 0.7996495962 	 F1_val: 0.6750362144
Epoch: 6 	 Batch: 0 	 Loss: 0.8652065992 	 F1_val: 0.6821852732
Epoch: 7 	 Batch: 0 	 Loss: 0.9100013971 	 F1_val: 0.6768642447
Epoch: 8 	 Batch: 0 	 Loss: 0.6142103076 	 F1_val: 0.6853080569
Epoch: 9 	 Batch: 0 	 Loss: 1.1335546970 	 F1_val: 0.6774500476
Epoch: 10 	 Batch: 0 	 Loss: 1.5742430687 	 F1_val: 0.6538461538
Epoch: 11 	 Batch: 0 	 Loss: 1.2565995455 	 F1_val: 0.6331811263
Epoch: 12 	 Batch: 0 	 Loss: 0.4229889810 	 F1_val: 0.6442211055
Epoch: 13 	 Batch: 0 	 Loss: 0.4202293158 	 F1_val: 0.6290983607
Epoch: 14 	 Batch: 0 	 Loss: 0.6392421126 	 F1_val: 0.6233367451
Epoch: 15 	 Batch: 0 	 Loss: 0.7861

# Convolutional Neural Network

In [68]:
def text2index(text, vocab):
    inds = []
    for t in range(len(text)):
    doc = []
    for word in text[t].split():
        try:
        doc.append(voc.index(word))
        except:
        pass
    inds.append(doc)
    
    return inds

In [69]:
voc = list(vectorizer.vocabulary_)
x_train = text2index(list(X_train)[:4550], voc)
x_test = text2index(list(X_test)[:3000], voc)

In [70]:
import torch
from torch import nn

In [88]:
batch_size = 50

x_train = [torch.tensor(x) for x in x_train]
X_train = nn.utils.rnn.pad_sequence(x_train, batch_first=True, padding_value=0).long()
X_train.shape
X_train = X_train.view(-1, batch_size, X_train.shape[1])

x_test = [torch.tensor(x) for x in x_test]
X_test = nn.utils.rnn.pad_sequence(x_test, batch_first=True, padding_value=0,).long()
X_test  = X_test.view(-1, batch_size, X_test.shape[1])

len_voc = (max(X_train.max(), X_test.max())+1).item()
type(len_voc)

Y_train = torch.tensor(list(y_train[:4550])).view(-1, batch_size)
Y_test = torch.tensor(list(y_test[:3000])).cuda()

/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [72]:
Y_train.shape

torch.Size([91, 50])

In [73]:
print(len_voc)

9680


In [100]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics

class Model(nn.Module):
    def __init__(self, len_voc):
        super(Model, self).__init__()
        self.drop = nn.Dropout(0.5)
        self.embeddings = nn.Embedding(len_voc, 100)
        self.cnn = nn.Conv2d(1, 100, (3, 100))
        self.clf = nn.Linear(100, 2)

    def forward(self, x):
        x = self.embeddings(x)
        x = x.unsqueeze(1)
        x = self.cnn(x)
        x = x.max(2)[0].squeeze(2)
        x = F.relu(x) 
        #x = self.drop(x)
        x = self.clf(x)
        return F.log_softmax(x, 1)

In [106]:
model = Model(len_voc).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterio  = nn.NLLLoss()

In [114]:
epochs = 20
LOSS = []
for e in range(epochs):
    for i, (x, y) in enumerate(zip(X_train, Y_train)):
        
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x)
        
        loss = criterio(y_pred, y)
        loss.backward()
        optimizer.step()
        
        LOSS.append(loss.item())
        if i%200==0:
            with torch.no_grad():
                f1 = get_f1(X_test, Y_test, model)
            print('Epoch: %d \t Batch: %d \t Loss: %.10f \t F1_val: %.10f'%(e,i, torch.tensor(LOSS[-100:]).mean(), f1))

Epoch: 0 	 Batch: 0 	 Loss: 0.7272968888 	 F1_val: 0.5854700855
Epoch: 1 	 Batch: 0 	 Loss: 0.6933086514 	 F1_val: 0.2840778923
Epoch: 2 	 Batch: 0 	 Loss: 0.6708566546 	 F1_val: 0.0676691729
Epoch: 3 	 Batch: 0 	 Loss: 0.6597657204 	 F1_val: 0.2624434389
Epoch: 4 	 Batch: 0 	 Loss: 0.6279397011 	 F1_val: 0.5860862806
Epoch: 5 	 Batch: 0 	 Loss: 0.6206777692 	 F1_val: 0.6181184669
Epoch: 6 	 Batch: 0 	 Loss: 0.5997678638 	 F1_val: 0.5831858407
Epoch: 7 	 Batch: 0 	 Loss: 0.5754185319 	 F1_val: 0.4136310223
Epoch: 8 	 Batch: 0 	 Loss: 0.5691158175 	 F1_val: 0.3873325213
Epoch: 9 	 Batch: 0 	 Loss: 0.5499871373 	 F1_val: 0.5446293495
Epoch: 10 	 Batch: 0 	 Loss: 0.5276023746 	 F1_val: 0.6330814442
Epoch: 11 	 Batch: 0 	 Loss: 0.5176726580 	 F1_val: 0.6438947782
Epoch: 12 	 Batch: 0 	 Loss: 0.5028102398 	 F1_val: 0.6443064182
Epoch: 13 	 Batch: 0 	 Loss: 0.4815537333 	 F1_val: 0.6117969822
Epoch: 14 	 Batch: 0 	 Loss: 0.4663649499 	 F1_val: 0.5905319668
Epoch: 15 	 Batch: 0 	 Loss: 0.4551

## Word2Vec Embedding

Instead of starting with random embedding, we can alternatively start with word2vec embedding 

In [79]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

import os
path_w2v=os.path.abspath('glove.twitter.27B.200d_w2v.txt') 
embed_lookup = KeyedVectors.load_word2vec_format(path_w2v) 


/srv/conda/envs/saturn/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [80]:
weights = torch.FloatTensor(embed_lookup.vectors)
embeddings = nn.Embedding.from_pretrained(weights, padding_idx=embed_lookup.vocab["<space>"].index)

In [116]:
class ModelW2V(nn.Module):
    def __init__(self, len_voc, init_embedding):
        super(ModelW2V, self).__init__()
        self.drop = nn.Dropout(0.5)
        self.embeddings = init_embedding
        self.embeddings.weight.requires_grad = False
        self.cnn1 = nn.Conv2d(1, 100, (3, 200))
        self.cnn2 = nn.Conv2d(1, 100, (4, 200))
        self.cnn3 = nn.Conv2d(1, 100, (5, 200))
        self.clf = nn.Linear(300, 2)

    def forward(self, x):
        x = self.embeddings(x)
        x = x.unsqueeze(1)
        #print(x.shape)

        x1 = self.cnn1(x)
        #print(x1.shape)
        x1 = x1.max(2)[0].squeeze(2)

        x2 = self.cnn2(x)
        x2 = x2.max(2)[0].squeeze(2)

        x3 = self.cnn3(x)
        x3 = x3.max(2)[0].squeeze(2)

        x = torch.cat([x1,x2,x3],1)

        x = F.relu(x) 
        #x = self.drop(x)
        x = self.clf(x)
        return F.log_softmax(x, 1)

In [117]:
model_w2v = ModelW2V(len_voc, embeddings).cuda()
optimizer = optim.Adam(model_w2v.parameters(), lr=1e-4)
criterio  = nn.NLLLoss()

In [118]:
print(X_train[0].shape)

torch.Size([50, 23])


In [119]:
model_w2v(X_train[0].cuda())

tensor([[-0.8539, -0.5547],
        [-0.9280, -0.5031],
        [-0.8436, -0.5624],
        [-0.7707, -0.6212],
        [-0.7900, -0.6049],
        [-0.6958, -0.6905],
        [-0.8271, -0.5751],
        [-0.7647, -0.6264],
        [-0.7614, -0.6293],
        [-0.7748, -0.6177],
        [-0.9176, -0.5100],
        [-0.7648, -0.6263],
        [-0.7852, -0.6089],
        [-0.7177, -0.6692],
        [-0.7667, -0.6246],
        [-0.7184, -0.6685],
        [-0.6969, -0.6894],
        [-0.8736, -0.5403],
        [-0.8047, -0.5928],
        [-0.8236, -0.5778],
        [-0.7453, -0.6436],
        [-0.8222, -0.5789],
        [-0.7259, -0.6614],
        [-0.8134, -0.5858],
        [-0.7539, -0.6359],
        [-0.7893, -0.6054],
        [-0.8425, -0.5632],
        [-0.7911, -0.6039],
        [-0.7230, -0.6641],
        [-0.7890, -0.6057],
        [-0.7584, -0.6319],
        [-0.7652, -0.6260],
        [-0.7874, -0.6070],
        [-0.7152, -0.6716],
        [-0.7693, -0.6224],
        [-0.7896, -0

In [122]:
epochs = 50
LOSS = []
for e in range(epochs):
    for i, (x, y) in enumerate(zip(X_train, Y_train)):
        
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        y_pred = model_w2v(x)
        
        loss = criterio(y_pred, y)
        loss.backward()
        optimizer.step()
        
        LOSS.append(loss.item())
        if i%200==0:
            with torch.no_grad():
                f1 = get_f1(X_test, Y_test, model_w2v)
            print('Epoch: %d \t Batch: %d \t Loss: %.10f \t F1_val: %.10f'%(e,i, torch.tensor(LOSS[-100:]).mean(), f1))

Epoch: 0 	 Batch: 0 	 Loss: 0.6944720745 	 F1_val: 0.5031446541
Epoch: 1 	 Batch: 0 	 Loss: 0.6624437571 	 F1_val: 0.4167610419
Epoch: 2 	 Batch: 0 	 Loss: 0.6180655360 	 F1_val: 0.5101721440
Epoch: 3 	 Batch: 0 	 Loss: 0.5795461535 	 F1_val: 0.5519677094
Epoch: 4 	 Batch: 0 	 Loss: 0.5419302583 	 F1_val: 0.5873786408
Epoch: 5 	 Batch: 0 	 Loss: 0.5047081709 	 F1_val: 0.6112423240
Epoch: 6 	 Batch: 0 	 Loss: 0.4683764577 	 F1_val: 0.6266173752
Epoch: 7 	 Batch: 0 	 Loss: 0.4332748353 	 F1_val: 0.6380255941
Epoch: 8 	 Batch: 0 	 Loss: 0.3996908069 	 F1_val: 0.6454834317
Epoch: 9 	 Batch: 0 	 Loss: 0.3679221869 	 F1_val: 0.6507004067
Epoch: 10 	 Batch: 0 	 Loss: 0.3381090164 	 F1_val: 0.6550335570
Epoch: 11 	 Batch: 0 	 Loss: 0.3103032410 	 F1_val: 0.6625332152
Epoch: 12 	 Batch: 0 	 Loss: 0.2845363617 	 F1_val: 0.6646072374
Epoch: 13 	 Batch: 0 	 Loss: 0.2607834637 	 F1_val: 0.6666666667
Epoch: 14 	 Batch: 0 	 Loss: 0.2390759438 	 F1_val: 0.6672512056
Epoch: 15 	 Batch: 0 	 Loss: 0.2193